In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
# Instantiate Spark Session
spark = (SparkSession
  .builder
  .appName("US_Accidents")
  .getOrCreate())
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/26 13:55:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
24/11/26 13:55:01 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.


In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    elif col == 'Traffic_Interference':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
# Undersampling
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("SeverityIndex").count()

# Step 2: Use PySpark's min() function to find the minimum count
min_class_size = class_counts.agg(F.min('count')).collect()[0][0]

undersampled_train_list = []

for row in class_counts.collect():
    class_label = row['SeverityIndex']
    class_size = row['count']

    if class_size > min_class_size:
        # Sample the data for this class to the size of the minimum class
        class_data = train.filter(F.col("SeverityIndex") == class_label)
        class_data_undersampled = class_data.sample(withReplacement=False, fraction=min_class_size / class_size)
    else:
        # For classes that are already at the minimum size, keep all samples
        class_data_undersampled = train.filter(F.col("SeverityIndex") == class_label)

    undersampled_train_list.append(class_data_undersampled)

# Combine all the undersampled DataFrames
undersampled_train = undersampled_train_list[0]  # start with the first one
for df in undersampled_train_list[1:]:
    undersampled_train = undersampled_train.union(df)

# Show the result
undersampled_train.show()

# Step 4: Group by 'Severity' and count the occurrences in the undersampled DataFrame
undersampled_class_counts = undersampled_train.groupBy("SeverityIndex").count()

# Show the result
undersampled_class_counts.show()

+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|Severity|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|SeverityIndex|
+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|       2|        -13|      

+-------------+-----+
|SeverityIndex|count|
+-------------+-----+
|          0.0|52048|
|          1.0|52053|
|          3.0|52151|
|          2.0|52194|
+-------------+-----+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

undersampled_train = assembler.transform(undersampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(undersampled_train)
scaledTrainData = scalerModel.transform(undersampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures',
                        #maxIter=10, 
                        #regParam=0.3, 
                        #elasticNetParam=0.8,
                        family="multinomial")

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 4
Coefficients: DenseMatrix([[-1.90929596e-02, -8.36946770e-02,  3.11084682e-02,
              -4.79608575e-02, -1.51375046e-02,  2.37053450e-03,
              -2.95503795e-02, -3.90660816e-02, -1.42996169e-02,
               3.99667630e-02,  5.30715374e-01,  3.37388195e-01,
               2.18248097e-02,  7.77989485e-04, -9.91838575e-02,
               2.41135709e-02,  3.21558833e-02,  8.11745315e-03,
               4.06794656e-02, -2.87539320e-02,  6.16266651e-02,
               8.39615666e-05],
             [-1.38522931e-01, -6.57579455e-02,  2.30960880e-01,
               7.56783305e-03,  1.34713821e-01,  6.98923695e-03,
              -6.33300458e-02, -2.07844936e-02,  4.31688697e-02,
               3.15530007e-02,  2.27130992e-01,  1.54521323e-01,
              -1.18121639e-01, -1.24993706e-01,  4.68440658e-01,
               5.46631146e-02, -1.33313799e-01,  2.14355370e-03,
               2.85228893e-01, -2.66216088e-01, -9.59826525e-02,
              

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

evaluator = MulticlassClassificationEvaluator(labelCol='SeverityIndex', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       3.0|
|       2.0|
+----------+



Accuracy: 0.3867268748915199


In [12]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.7679468934981271
Recall: 0.3867268748915199
F1 Score: 0.47738835068579244


In [13]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN SeverityIndex = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('SeverityIndex').agg(
    (F.sum('is_correct') / F.count('SeverityIndex')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+-------------+-------------------+
|SeverityIndex|           accuracy|
+-------------+-------------------+
|          0.0|0.35872981927151865|
|          1.0| 0.4986245940334002|
|          3.0| 0.7780771303209915|
|          2.0|0.42515772430350063|
+-------------+-------------------+



In [14]:
# Calculate metrics by class
labels = lrPred.select('SeverityIndex').distinct().orderBy('SeverityIndex').rdd.flatMap(lambda x: x).collect()

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0.0 - Precision: 0.8874, Recall: 0.3587, F1 Score: 0.5109
Class 1.0 - Precision: 0.3229, Recall: 0.4986, F1 Score: 0.3920
Class 2.0 - Precision: 0.0623, Recall: 0.4252, F1 Score: 0.1086
Class 3.0 - Precision: 0.0284, Recall: 0.7781, F1 Score: 0.0548
